In [2]:
%run 'C:\dev\prod\metrikaHelper.py'
import psycopg2
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[2], 'metrikapassword': row[3]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}

In [3]:
import pandas as pd
def getMetrikaData (project,rowCount):
    VipProjectInfo = getVipProjectByDomain(project)
    print VipProjectInfo
    metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo[rowCount]['metrikalogin'],
                                                     'yandexPassword':VipProjectInfo[rowCount]['metrikapassword'],
                                                     'domain':VipProjectInfo[rowCount]['domain']})

    print metrikaCredentials
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo[rowCount]['projectid'])])
    if project=='cb39.ru':
        goalsMetrics = goalsMetrics.replace('ym:s:goal19175290visits,','')
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions='ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMTerm,ym:s:UTMContent'
    date1 = '2016-04-26'
    date2 = '2016-05-25'
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    params = '&'.join(['dimensions={0}'.format(dimensions),
                       'metrics={0}'.format(metrics),
                       'date1={0}'.format(date1),
                      'date2={0}'.format(date2),
                      'ids={0}'.format(counterId),
                      'oauth_token={0}'.format(token),
                       'accuracy=full',
                      'limit=100000',
                      'offset=1',
                      "filters=ym:s:UTMMedium=='adwords'"])
    url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
    print url
    df = pd.read_csv(url,encoding='utf-8')
    df.rename(columns={u'\ufeff"UTM Source"': 'UTM Source',
                            u'Визиты': 'visits'
                           }, inplace=True)
    for i in xrange(0,len(df)):
        df.loc[i,'goal_visits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
    df['domain'] = VipProjectInfo[0]['domain']
    df.drop([column for column in df.columns if u'Целевые визиты' in column], axis=1, inplace=True)
    df = df[~df[u'UTM Source'].isin([u'Итого и средние'])]
    return df

In [4]:
projects = ['academoffice.ru','accessoriesforhome.ru','adel-omsk.ru','advocates.su','a-e-test.com','alcotek.ru','alpinabook.ru','altpremium.ru','apexair.ru','api-test','argus-x.ru','arhivcentr.ru',
'asiavc.ru','baxburo.ru','cb39.ru','cesis.ru','cvetkrasok.ru','darion-svet.ru','del-ko.ru','di-ci.ru','doorlock.ru','edemdom.ru','ferkon.ru','fosterland.ru','gemainvest.ru','guestworkers.ru',
'gyroscooter.ru','hotelsipk.ru','hp-brakes.ru','kerasol.ru','labotini.ru','lodki-piter.ru','medstrahovka.ru','milo-kormilo.ru','mlprussia.com','multifin.ru','m-y.ru','nasoscentr.ru',
'okna-okoshkino.ru','optikdeal.ru','organickids.ru','ot-mebel.ru','perchini.ru','pickup-center.ru','pinakotekaresto.com','prosto-gruz.ru','proudmom.ru','realstroy.com','ribambelle.ru',
'rin-tek.ru','rusmaneken.ru','russj.ru','severingroup.ru','simple-pro.com','sitlos.ru','skylift-nsk.com','stemphys.ru','stm66.ru','stranao3.ru','svetlux.ru','syntone.ru','tencon.ru',
'teploppu.ru','tiendti.ru','trans-gear.ru','trelaxorto.ru','vmstar.ru','vtb24leasing.ru','www.fabslog.ru','zbbb.ru']

In [5]:
total_df = pd.DataFrame()
for project in projects:
    print project
    try:
        if project == 'argus-x.ru':
            rowCount = 1
        else:
            rowCount = 0
        total_df = pd.concat([total_df,getMetrikaData(project,rowCount)])
    except:
        print project, 'exception'

academoffice.ru
[{'metrikalogin': 'academoff', 'projectid': 2383389L, 'domain': 'academoffice.ru', 'metrikapassword': 'academ2000'}]
{'Token': u'5cc9948767794035af9ca25479414560', 'counterID': [32446210]}
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMTerm,ym:s:UTMContent&metrics=ym:s:visits,ym:s:goal18202220visits,ym:s:goal16444500visits&date1=2016-04-26&date2=2016-05-25&ids=32446210&oauth_token=5cc9948767794035af9ca25479414560&accuracy=full&limit=100000&offset=1&filters=ym:s:UTMMedium=='adwords'
accessoriesforhome.ru
[{'metrikalogin': 'RookeeXML@yandex.ru', 'projectid': 2436833L, 'domain': 'accessoriesforhome.ru', 'metrikapassword': 'pass-RookeeXML'}]
{'Token': u'71e4f97915d34895b4de3b45dbb6beef', 'counterID': [18803587]}
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMTerm,ym:s:UTMContent&metrics=ym:s:visits,ym:s:goal18388385visits&date1=2016-0

In [6]:
total_df.to_excel('C:\\Users\\aleksey.makarov\\Desktop\\2016-05\\2016-05-26-AdWords_SNDS_divide.xlsx')

In [87]:
import requests

In [88]:
r = requests.get('https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D1%82%D0%BA%D0%BB%D0%BE%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5')

In [89]:
from bs4 import BeautifulSoup

In [90]:
soup = BeautifulSoup(r.text,'lxml')

In [99]:
len(soup.find_all(attrs={'id':'bodyContent'})[0].get_text().split(' '))

855

In [100]:
r = requests.get('https://en.wikipedia.org/wiki/Standard_deviation')

In [101]:
soup = BeautifulSoup(r.text,'lxml')

In [102]:
len(soup.find_all(attrs={'id':'bodyContent'})[0].get_text().split(' '))

6180

In [105]:
6180.0/855.0

7.228070175438597

In [106]:
r = requests.get('https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D1%82%D0%BA%D0%BB%D0%BE%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5')

In [110]:
soup = BeautifulSoup(r.text,'lxml')
import re
a = u

In [115]:
en_link = 'https:'+soup.find('a',attrs={'hreflang':'en','title':re.compile(u"английский")})['href']

In [116]:
en_link

'https://en.wikipedia.org/wiki/Standard_deviation'

In [118]:
requests.get(en_link).text

u'<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta charset="UTF-8"/>\n<title>Standard deviation - Wikipedia, the free encyclopedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Standard_deviation","wgTitle":"Standard deviation","wgCurRevisionId":714308184,"wgRevisionId":714308184,"wgArticleId":27590,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with unsourced statements","Articles with unsourced statements from November 2015","Articles to be split from April 2013","All articles to be split","Articles with unsourced statements from July 2012","Articles with unsourced statements from January 2012","All article

In [120]:
getEnglishVersionLink(soup)

'https://en.wikipedia.org/wiki/Standard_deviation'

In [121]:
url = 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0'

In [122]:
r = requests.get(url)

In [123]:
soup = BeautifulSoup(r.text,'lxml')

In [125]:
soup = getSoupByUrl(url)

In [127]:
getEnglishVersionLink(soup)

'https://en.wikipedia.org/wiki/Mathematical_statistics'

In [136]:
getWikipediaArticleLength(soup)

908

In [234]:
url = 'https://ru.wikipedia.org/wiki/%D0%91%D1%83%D0%BB%D0%B5%D0%B0%D0%BD'
comparisonData = getComparisonData(url)
print comparisonData[2],comparisonData[3]
print 'Rus: {0}; Eng: {1}; Ratio: {2}'.format(comparisonData[0],comparisonData[1],(float(comparisonData[1])/float(comparisonData[0])))

Булеан Power set
Rus: 253; Eng: 1820; Ratio: 7.19367588933


In [202]:
url = 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%BA%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7'
soup = getSoupByUrl(url)

In [213]:
getWikipediaPageLinks(soup)

[<a href="/wiki/%D0%A1%D0%BF%D0%B8%D1%80%D0%BC%D0%B5%D0%BD,_%D0%A7%D0%B0%D1%80%D0%BB%D1%8C%D0%B7_%D0%AD%D0%B4%D0%B2%D0%B0%D1%80%D0%B4" title="Спирмен, Чарльз Эдвард">Спирмен</a>,
 <a href="/wiki/%D0%9A%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D1%8F" title="Корреляция">корреляций</a>,
 <a href="/wiki/%D0%AD%D0%BA%D0%BE%D0%BD%D0%BE%D0%BC%D0%B8%D0%BA%D0%B0" title="Экономика">экономике</a>,
 <a href="/wiki/Statistica" title="Statistica">Statistica</a>,
 <a href="/wiki/%D0%9F%D0%BE%D0%BB%D0%B8%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F" title="Политология">политологии</a>,
 <a href="/wiki/%D0%93%D0%B0%D0%BB%D1%8C%D1%82%D0%BE%D0%BD,_%D0%A4%D1%80%D1%8D%D0%BD%D1%81%D0%B8%D1%81" title="Гальтон, Фрэнсис">Гальтоном</a>,
 <a href="/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BC%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BF%D1%80%D0%B0%D0%B2%D0%B4%D0%BE%D0%BF%D0%BE%D0%B4%D0%BE%D0%B1%D0%B8%D1%8F" title="Метод максимального правдоподобия">метод максимального правдоп

In [226]:
links = get2LevelWikiLinks(url)

In [232]:
getFirstHeading(soup).get_text()

u'\u041f\u0441\u0438\u0445\u043e\u043b\u043e\u0433\u0438\u044f'

In [235]:
def getEnglishVersionLink(soup):
    return 'https:'+soup.find('a',attrs={'hreflang':'en','title':re.compile(u"английский")})['href']
def getSoupByUrl(url):
    return BeautifulSoup(requests.get(url).text,'lxml')
def getWikipediaArticleLength(soup):
    return len(soup.find_all(attrs={'id':'bodyContent'})[0].get_text().split(' '))
def getComparisonData(russian_url):
    russian_soup = getSoupByUrl(russian_url)
    russian_title = getFirstHeading(russian_soup).get_text()
    english_url = getEnglishVersionLink(russian_soup)
    english_soup = getSoupByUrl(english_url)
    english_title = getFirstHeading(english_soup).get_text()
    return getWikipediaArticleLength(russian_soup),getWikipediaArticleLength(english_soup),russian_title,english_title
def getWikipediaPageLinks(soup):
    ignored_titles = [u'Поиск по категориям',u'Место, где можно получить справку',u'Список недавно созданных страниц',u'Форум участников Википедии',
    u'О проекте, о том, чем здесь можно заниматься, а также — где что находится',u'Сообщить об ошибке в этой статье',u'Статьи о текущих событиях в мире',u'Статьи, считающиеся лучшими статьями проекта',
    u'Поиск по двум первым буквам',u'Улучшение статьи',u'Просмотр этого шаблона']
    links = [tag for tag in soup.find_all('a',attrs={'href':re.compile(u"^/wiki"),'class':'','accesskey':'','title':re.compile(u"^(?!(Википедия:|Шаблон:|Портал:|Категория:|Служебная:|Обсуждение шаблона:|Проект:))")}) if tag['title'] not in ignored_titles]
    return list(set(links))
def createRuWikiLink(link):
    return 'https://ru.wikipedia.org'+link['href']
def getFirstHeading(soup):
    return soup.find('h1',attrs={'id':'firstHeading'})
def get2LevelWikiLinks(url):
    soup = getSoupByUrl(url)
    links = getWikipediaPageLinks(soup)
    first_links = [createRuWikiLink(link) for link in links]
    second_links = []
    for link in first_links:
        soup = getSoupByUrl(link)
        links = getWikipediaPageLinks(soup)
        second_links += [createRuWikiLink(link) for link in links]
    return list(set(second_links+first_links))

In [237]:
data = []
url = 'https://ru.wikipedia.org/wiki/%D0%91%D1%83%D0%BB%D0%B5%D0%B0%D0%BD'
links = get2LevelWikiLinks(url)
for link in links[:100]:
    print link
    try:
        comparisonData = getComparisonData(link)
        d = {'rus':comparisonData[2],'eng':comparisonData[3],'rus_length':comparisonData[0],'eng_length':comparisonData[1],'ratio':(float(comparisonData[1])/float(comparisonData[0])),'url':link}
    except TypeError:
        d = {'url':link}
    data.append(d)

https://ru.wikipedia.org/wiki/%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0
https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D1%80%D1%83%D0%BA%D1%82%D1%83%D1%80%D0%BD%D0%B0%D1%8F_%D0%B8%D0%BD%D0%B4%D1%83%D0%BA%D1%86%D0%B8%D1%8F
https://ru.wikipedia.org/wiki/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D1%80_%D0%BF%D0%BE%D0%B4%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2
https://ru.wikipedia.org/wiki/%D0%91%D0%B8%D0%B5%D0%BA%D1%86%D0%B8%D1%8F
https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2
https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_(%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B9)
https://ru.wikipedia.org/wiki/%D0%A4%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BB%D0%BE%D0%B3%D0%B8%D0%BA%D0%B0
https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%BC%D0%BD

In [245]:
print pd.DataFrame(data).loc[96,'url']

https://ru.wikipedia.org/wiki/%D0%9C%D0%BE%D1%89%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2%D0%B0


In [247]:
pd.DataFrame(data).to_excel('D:\\wiki_research.xlsx')

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:845: UserWarning: Ignoring URL 'https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%BE%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BE%D1%82%D0%BD%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BC%D0%B5%D0%B6%D0%B4%D1%83_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D0%B5%D0%BC_%D0%B8_%D0%BE%D0%B1%D1%8A%D1%91%D0%BC%D0%BE%D0%BC_%D0%BF%D0%BE%D0%BD%D1%8F%D1%82%D0%B8%D1%8F' > 255 characters since it exceeds Excel's limit for URLS
  "Excel's limit for URLS" % url)
